# **ENTRENAMIENTO DEL MODELO DE DETECCIÓN DE PERSONAS - YOLOv11**

# *1. Configuración Inicial y Preparación*

In [1]:
# Instalación de dependencias
!pip install ultralytics
!pip install wandb
!pip install seaborn
!pip install scikit-learn
!pip install tensorboard

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 117.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 82.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 64.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 102.7 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstallin

In [2]:
# Importar librerías necesarias
import os
import yaml
import torch
import numpy as np
import wandb
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from datetime import datetime
from sklearn.metrics import confusion_matrix, classification_report, precision_recall_curve, roc_curve, auc
from google.colab import drive
import wandb
from ultralytics import YOLO
import cv2
from IPython.display import clear_output, Image, display
import glob
import shutil
import random
import gc
import json
import time

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [12]:
from google.colab import drive
# Montar Google Drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
# Configuración de rutas
BASE_PATH = '/content/drive/MyDrive/Proyecto-IA3'
DATASET_PATH = f'/content/drive/MyDrive/dataset_people'
MODELS_PATH = f'{BASE_PATH}/models'
CHECKPOINTS_PATH = f'{BASE_PATH}/checkpoints'
RESULTS_PATH = f'{BASE_PATH}/results'
LOGS_PATH = f'{BASE_PATH}/logs'


# Crear directorios necesarios
for path in [MODELS_PATH, CHECKPOINTS_PATH, RESULTS_PATH, LOGS_PATH]:
    os.makedirs(path, exist_ok=True)

# Configuración de semillas para reproducibilidad
def set_seeds(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    os.environ['PYTHONHASHSEED'] = str(seed)

set_seeds(42)

In [5]:
# Verificar GPU disponible
!nvidia-smi
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
print(f"CUDA device: {torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'None'}")

Sun May  4 06:22:17 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   47C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

# **2. Análisis y Visualización del Dataset**

In [ ]:
# Función para validar el dataset
def validate_dataset(dataset_path):
    """Valida la integridad del dataset, verificando imágenes y etiquetas."""
    errors = []
    for split in ['train', 'val', 'test']:
        img_path = os.path.join(dataset_path, 'images', split)
        label_path = os.path.join(dataset_path, 'labels', split)

        if not os.path.exists(img_path) or not os.path.exists(label_path):
            errors.append(f"Directorio faltante: {img_path} o {label_path}")
            continue

        img_files = [f for f in os.listdir(img_path) if f.endswith('.jpg')]
        label_files = [f for f in os.listdir(label_path) if f.endswith('.txt')]

        # Verificar correspondencia entre imágenes y etiquetas
        for img_file in img_files:
            label_file = img_file.replace('.jpg', '.txt')
            if label_file not in label_files:
                errors.append(f"Etiqueta faltante para {img_file} en {split}")

        # Validar contenido de etiquetas
        for label_file in label_files:
            with open(os.path.join(label_path, label_file), 'r') as f:
                for line in f.readlines():
                    parts = line.strip().split()
                    if len(parts) != 5:
                        errors.append(f"Formato inválido en {label_file}: {line}")
                        continue
                    class_id, x_center, y_center, width, height = map(float, parts)
                    if class_id != 0 or not (0 <= x_center <= 1 and 0 <= y_center <= 1 and 0 < width <= 1 and 0 < height <= 1):
                        errors.append(f"Valores inválidos en {label_file}: {line}")

    return errors

# Ejecutar validación del dataset
validation_errors = validate_dataset(DATASET_PATH)
if validation_errors:
    print("Errores en el dataset:")
    for error in validation_errors:
        print(f"- {error}")
else:
    print("Dataset validado correctamente.")

Dataset validado correctamente.


In [11]:
# Función para analizar el dataset
def analyze_dataset(dataset_path):
    """Analiza la estructura y estadísticas del dataset."""
    stats = {
        'train': {'images': 0, 'labels': 0, 'boxes': 0},
        'val': {'images': 0, 'labels': 0, 'boxes': 0},
        'test': {'images': 0, 'labels': 0, 'boxes': 0}
    }

    for split in ['train', 'val', 'test']:
        # Contar imágenes
        img_path = os.path.join(dataset_path, 'images', split)
        if os.path.exists(img_path):
            stats[split]['images'] = len([f for f in os.listdir(img_path) if f.endswith('.jpg')])

        # Contar labels y boxes
        label_path = os.path.join(dataset_path, 'labels', split)
        if os.path.exists(label_path):
            label_files = [f for f in os.listdir(label_path) if f.endswith('.txt')]
            stats[split]['labels'] = len(label_files)
            total_boxes = 0
            for label_file in label_files:
                with open(os.path.join(label_path, label_file), 'r') as f:
                    total_boxes += len(f.readlines())
            stats[split]['boxes'] = total_boxes

    return stats


# Visualizar estadísticas del dataset
dataset_stats = analyze_dataset(DATASET_PATH)
print("Dataset Statistics:")
print("-" * 50)
for split, stat in dataset_stats.items():
    print(f"{split.upper()}:")
    print(f"  Images: {stat['images']}")
    print(f"  Labels: {stat['labels']}")
    print(f"  Total Boxes: {stat['boxes']}")
    print(f"  Avg Boxes/Image: {stat['boxes']/stat['images'] if stat['images'] > 0 else 0:.2f}")
    print()

# Visualizar muestras del dataset
def visualize_dataset_samples(dataset_path, num_samples=9):
    """Visualiza muestras aleatorias del dataset con sus anotaciones."""
    train_images_path = os.path.join(dataset_path, 'images', 'train')
    train_labels_path = os.path.join(dataset_path, 'labels', 'train')

    image_files = [f for f in os.listdir(train_images_path) if f.endswith('.jpg')]
    sample_files = random.sample(image_files, min(num_samples, len(image_files)))

    fig, axes = plt.subplots(3, 3, figsize=(15, 15))
    axes = axes.ravel()

    for idx, img_file in enumerate(sample_files):
        img_path = os.path.join(train_images_path, img_file)
        label_path = os.path.join(train_labels_path, img_file.replace('.jpg', '.txt'))

        img = cv2.imread(img_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

        if os.path.exists(label_path):
            with open(label_path, 'r') as f:
                for line in f.readlines():
                    class_id, x_center, y_center, width, height = map(float, line.strip().split())
                    h, w = img.shape[:2]
                    x1 = int((x_center - width/2) * w)
                    y1 = int((y_center - height/2) * h)
                    x2 = int((x_center + width/2) * w)
                    y2 = int((y_center + height/2) * h)
                    cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)
                    cv2.putText(img, 'Person', (x1, y1-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

        axes[idx].imshow(img)
        axes[idx].axis('off')
        axes[idx].set_title(f'Sample {idx+1}')

    plt.tight_layout()
    plt.savefig(f'{RESULTS_PATH}/dataset_samples.png', dpi=300, bbox_inches='tight')
    plt.show()

visualize_dataset_samples(DATASET_PATH)

Output hidden; open in https://colab.research.google.com to view.

# **3. Transfer Learning (Fase 1)**

In [6]:
# Cargar el modelo preentrenado
print("Cargando yolo11n.pt...")
model = YOLO('yolo11n.pt')

Cargando yolo11n.pt...


100%|██████████| 5.35M/5.35M [00:00<00:00, 108MB/s]


In [ ]:
model

YOLO(
  (model): DetectionModel(
    (model): Sequential(
      (0): Conv(
        (conv): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(16, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (1): Conv(
        (conv): Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (2): C3k2(
        (cv1): Conv(
          (conv): Conv2d(32, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
          (act): SiLU(inplace=True)
        )
        (cv2): Conv(
          (conv): Conv2d(48, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(64, eps=0.001, momentum=0.03, affine=True, track_running_

In [13]:
# Configuración para Transfer Learning
def transfer_learning_phase():
    """Fase 1: Transfer Learning con backbone congelado."""

    # Configurar la clave API de WandB
    try:
        with open('/content/drive/MyDrive/Proyecto-IA3/wandb_api_key.txt', 'r') as f:
            os.environ["WANDB_API_KEY"] = f.read().strip()
    except FileNotFoundError:
        print("Archivo wandb_api_key.txt no encontrado. Usando clave de respaldo (no recomendado para código compartido).")
        os.environ["WANDB_API_KEY"] = "30a7a34eb742653f47d9208966cbed1dc6f31e2d"

    # Inicializar WandB
    wandb.init(
        project="school-violence-detection-yolo11_V2",
        name=f"transfer_learning_{datetime.now().strftime('%Y%m%d_%H%M%S')}",
        config={
            "phase": "transfer_learning",
            "architecture": "YOLOv11n",
            "frozen_layers": 10,
            "epochs": 30,
            "batch_size": 16,
            "learning_rate": 0.001,
        }
    )

    # Cargar el modelo preentrenado
    print("Cargando yolo11n.pt...")
    model = YOLO('yolo11n.pt')

    # Contador para rastrear épocas
    current_epoch = [0]

    # Callback para imprimir métricas por época
    def on_val_end(validator):
        """Imprime métricas después de cada validación (por época)."""
        current_epoch[0] += 1  # Incrementar contador de épocas

        # Obtener métricas del validator
        metrics = validator.metrics

        # Para YOLOv11, las métricas están en el objeto metrics
        if hasattr(metrics, 'box'):
            print(f"\n=== Época {current_epoch[0]}/30 ===")
            print(f"  Precisión: {metrics.box.mp:.4f}")  # mean precision
            print(f"  Recall: {metrics.box.mr:.4f}")     # mean recall
            print(f"  mAP@50: {metrics.box.map50:.4f}")  # mAP at IoU 0.5
            print(f"  mAP@50-95: {metrics.box.map:.4f}") # mAP at IoU 0.5:0.95
        else:
            print(f"\n=== Época {current_epoch[0]}/30 ===")
            print("  Métricas aún no disponibles")

    # Configuración de hiperparámetros
    transfer_params = {
        'data': f'{DATASET_PATH}/data2.yaml',
        'epochs': 30,
        'batch': 16,
        'imgsz': 640,
        'freeze': 10,
        'optimizer': 'AdamW',
        'lr0': 0.001,
        'lrf': 0.1,
        'momentum': 0.937,
        'weight_decay': 0.0005,
        'warmup_epochs': 3,
        'warmup_momentum': 0.8,
        'warmup_bias_lr': 0.1,
        'box': 7.5,
        'cls': 0.5,
        'dfl': 1.5,
        'save': True,
        'save_period': 5,
        'cache': False,  # Desactivado debido a falta de espacio en disco
        'device': 0,
        'workers': 4,
        'project': f'{MODELS_PATH}/transfer_learning',
        'name': 'exp',
        'exist_ok': True,
        'pretrained': True,
        'verbose': True,
        'seed': 42,
        'deterministic': True,
        'single_cls': True,
        'cos_lr': True,
        'patience': 10,
        'amp': True,
        'plots': True,
    }

    print("Iniciando Transfer Learning con backbone congelado...")
    print(f"Capas congeladas: {transfer_params['freeze']}")

    # Registrar callback
    model.add_callback('on_val_end', on_val_end)

    # Medir tiempo de entrenamiento
    start_time = time.time()

    # Entrenar
    results = model.train(**transfer_params)

    # Calcular tiempo de entrenamiento
    training_time = time.time() - start_time

    # Guardar el modelo
    model.save(f'{CHECKPOINTS_PATH}/transfer_learning_model.pt')

    # Evaluar el modelo
    metrics = model.val()

    # Guardar métricas
    transfer_metrics = {
        'mAP50': float(metrics.box.map50),
        'mAP50-95': float(metrics.box.map),
        'precision': float(metrics.box.mp),
        'recall': float(metrics.box.mr),
        'training_time': training_time,
    }

    with open(f'{RESULTS_PATH}/transfer_learning_metrics.json', 'w') as f:
        json.dump(transfer_metrics, f, indent=4)

    # Visualizar curvas de entrenamiento
    plot_training_curves(results, phase='transfer_learning')

    # Limpiar memoria
    gc.collect()
    torch.cuda.empty_cache()

    wandb.finish()

    return model, results, metrics



# Función corregida para visualizar curvas de entrenamiento
def plot_training_curves(results, phase='transfer_learning'):
    """Visualiza las curvas de entrenamiento (pérdida, precisión, recall, mAP)."""

    try:
        # Intentar leer el archivo CSV generado por YOLO
        results_csv = f'{MODELS_PATH}/transfer_learning/exp/results.csv'
        if os.path.exists(results_csv):
            df = pd.read_csv(results_csv)

            fig, axes = plt.subplots(2, 2, figsize=(15, 10))

            # Graficar pérdidas
            if 'train/box_loss' in df.columns and 'val/box_loss' in df.columns:
                axes[0, 0].plot(df['epoch'], df['train/box_loss'], label='Train Box Loss')
                axes[0, 0].plot(df['epoch'], df['val/box_loss'], label='Val Box Loss')
                axes[0, 0].set_title('Box Loss')
                axes[0, 0].set_xlabel('Epochs')
                axes[0, 0].set_ylabel('Loss')
                axes[0, 0].legend()
                axes[0, 0].grid(True)

            if 'train/cls_loss' in df.columns and 'val/cls_loss' in df.columns:
                axes[0, 1].plot(df['epoch'], df['train/cls_loss'], label='Train Class Loss')
                axes[0, 1].plot(df['epoch'], df['val/cls_loss'], label='Val Class Loss')
                axes[0, 1].set_title('Classification Loss')
                axes[0, 1].set_xlabel('Epochs')
                axes[0, 1].set_ylabel('Loss')
                axes[0, 1].legend()
                axes[0, 1].grid(True)

            # Graficar métricas
            if 'metrics/precision(B)' in df.columns and 'metrics/recall(B)' in df.columns:
                axes[1, 0].plot(df['epoch'], df['metrics/precision(B)'], label='Precision')
                axes[1, 0].plot(df['epoch'], df['metrics/recall(B)'], label='Recall')
                axes[1, 0].set_title('Precision & Recall')
                axes[1, 0].set_xlabel('Epochs')
                axes[1, 0].set_ylabel('Value')
                axes[1, 0].legend()
                axes[1, 0].grid(True)

            if 'metrics/mAP50(B)' in df.columns and 'metrics/mAP50-95(B)' in df.columns:
                axes[1, 1].plot(df['epoch'], df['metrics/mAP50(B)'], label='mAP@50')
                axes[1, 1].plot(df['epoch'], df['metrics/mAP50-95(B)'], label='mAP@50-95')
                axes[1, 1].set_title('mAP Metrics')
                axes[1, 1].set_xlabel('Epochs')
                axes[1, 1].set_ylabel('mAP')
                axes[1, 1].legend()
                axes[1, 1].grid(True)

            plt.tight_layout()
            plt.savefig(f'{RESULTS_PATH}/{phase}_training_curves.png', dpi=300, bbox_inches='tight')
            plt.show()
            print(f"Gráficas guardadas en: {RESULTS_PATH}/{phase}_training_curves.png")

        else:
            print(f"No se encontró el archivo de resultados en: {results_csv}")

    except Exception as e:
        print(f"Error al generar gráficas: {e}")()




# Función alternativa para extraer métricas del archivo CSV generado por YOLO
def get_training_metrics(results_csv_path=None):
    """Extrae métricas del archivo results.csv generado durante el entrenamiento."""
    if results_csv_path is None:
        results_csv_path = f'{MODELS_PATH}/transfer_learning/exp/results.csv'

    if os.path.exists(results_csv_path):
        df = pd.read_csv(results_csv_path)
        # Mostrar las últimas métricas
        if len(df) > 0:
            last_row = df.iloc[-1]
            print("\nMétricas finales del entrenamiento:")
            print(f"Época: {last_row['epoch']}")
            print(f"mAP@50: {last_row['metrics/mAP50(B)']:.4f}")
            print(f"mAP@50-95: {last_row['metrics/mAP50-95(B)']:.4f}")
            print(f"Precisión: {last_row['metrics/precision(B)']:.4f}")
            print(f"Recall: {last_row['metrics/recall(B)']:.4f}")
        return df
    else:
        print(f"No se encontró el archivo de resultados en {results_csv_path}")
        return None



In [14]:
# Ejecutar Transfer Learning
print("Iniciando fase de Transfer Learning...")
transfer_model, transfer_results, transfer_metrics = transfer_learning_phase()


# Mostrar métricas finales
if os.path.exists(f'{RESULTS_PATH}/transfer_learning_metrics.json'):
    with open(f'{RESULTS_PATH}/transfer_learning_metrics.json', 'r') as f:
        metrics = json.load(f)
        print("\nMétricas finales de Transfer Learning:")
        print(f"mAP@50: {metrics['mAP50']:.4f}")
        print(f"mAP@50-95: {metrics['mAP50-95']:.4f}")
        print(f"Precisión: {metrics['precision']:.4f}")
        print(f"Recall: {metrics['recall']:.4f}")
        print(f"Tiempo de entrenamiento: {metrics['training_time']:.2f} segundos")

# Verificar si el modelo se guardó correctamente
if os.path.exists(f'{CHECKPOINTS_PATH}/transfer_learning_model.pt'):
    print(f"\nModelo guardado exitosamente en: {CHECKPOINTS_PATH}/transfer_learning_model.pt")
else:
    print("\nAdvertencia: No se encontró el archivo del modelo guardado")

# Extraer métricas adicionales del archivo CSV si está disponible
print("\nIntentando extraer métricas adicionales del archivo CSV...")
csv_metrics = get_training_metrics()
if csv_metrics is not None:
    print("Métricas extraídas exitosamente del archivo CSV")

Iniciando fase de Transfer Learning...


Cargando yolo11n.pt...
Iniciando Transfer Learning con backbone congelado...
Capas congeladas: 10
Ultralytics 8.3.126 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=True, cutmix=0.0, data=/content/drive/MyDrive/dataset_people/data2.yaml, degrees=0.0, deterministic=True, device=cuda:0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=30, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=10, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.001, lrf=0.1, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=exp, nbs=64, nms=False, opset=None, o

100%|██████████| 755k/755k [00:00<00:00, 25.6MB/s]

Overriding class names with single class.
Overriding model.yaml nc=80 with nc=1

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      6640  ultralytics.nn.modules.block.C3k2            [32, 64, 1, False, 0.25]      
  3                  -1  1     36992  ultralytics.nn.modules.conv.Conv             [64, 64, 3, 2]                
  4                  -1  1     26080  ultralytics.nn.modules.block.C3k2            [64, 128, 1, False, 0.25]     
  5                  -1  1    147712  ultralytics.nn.modules.conv.Conv             [128, 128, 3, 2]              
  6                  -1  1     87040  ultralytics.nn.modules.block.C3k2            [128, 128, 1, True]           
  7    

YOLO11n summary: 181 layers, 2,590,035 parameters, 2,590,019 gradients, 6.4 GFLOPs

Transferred 448/499 items from pretrained weights
Freezing layer 'model.0.conv.weight'
Freezing layer 'model.0.bn.weight'
Freezing layer 'model.0.bn.bias'
Freezing layer 'model.1.conv.weight'
Freezing layer 'model.1.bn.weight'
Freezing layer 'model.1.bn.bias'
Freezing layer 'model.2.cv1.conv.weight'
Freezing layer 'model.2.cv1.bn.weight'
Freezing layer 'model.2.cv1.bn.bias'
Freezing layer 'model.2.cv2.conv.weight'
Freezing layer 'model.2.cv2.bn.weight'
Freezing layer 'model.2.cv2.bn.bias'
Freezing layer 'model.2.m.0.cv1.conv.weight'
Freezing layer 'model.2.m.0.cv1.bn.weight'
Freezing layer 'model.2.m.0.cv1.bn.bias'
Freezing layer 'model.2.m.0.cv2.conv.weight'
Freezing layer 'model.2.m.0.cv2.bn.weight'
Freezing layer 'model.2.m.0.cv2.bn.bias'
Freezing layer 'model.3.conv.weight'
Freezing layer 'model.3.bn.weight'
Freezing layer 'model.3.bn.bias'
Freezing layer 'model.4.cv1.conv.weight'
Freezing layer 'mo

...

                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 50/50 [00:22<00:00,  2.27it/s]


                   all       1600       4251      0.957      0.954      0.988      0.763

=== Época 31/30 ===
  Precisión: 0.9573
  Recall: 0.9537
  mAP@50: 0.9882
  mAP@50-95: 0.7632
Speed: 0.3ms preprocess, 2.6ms inference, 0.0ms loss, 3.1ms postprocess per image
Results saved to /content/drive/MyDrive/Proyecto-IA3/models/transfer_learning/exp
Ultralytics 8.3.126 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
YOLO11n summary (fused): 100 layers, 2,582,347 parameters, 0 gradients, 6.3 GFLOPs
val: Fast image access ✅ (ping: 0.5±0.1 ms, read: 14.8±8.1 MB/s, size: 98.1 KB)


val: Scanning /content/drive/.shortcut-targets-by-id/1EKy5MKJmoLNQxoFyuOzjyUbOAB4YbrrM/dataset_people/labels/val.cache... 1600 images, 0 backgrounds, 0 corrupt: 100%|██████████| 1600/1600 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 100/100 [00:42<00:00,  2.35it/s]


                   all       1600       4251      0.958      0.954      0.988      0.763

=== Época 32/30 ===
  Precisión: 0.9579
  Recall: 0.9538
  mAP@50: 0.9882
  mAP@50-95: 0.7633
Speed: 0.4ms preprocess, 4.1ms inference, 0.0ms loss, 1.7ms postprocess per image
Results saved to /content/drive/MyDrive/Proyecto-IA3/models/transfer_learning/exp
No hay datos de resultados para visualizar



Métricas finales de Transfer Learning:
mAP@50: 0.9882
mAP@50-95: 0.7633
Precisión: 0.9579
Recall: 0.9538
Tiempo de entrenamiento: 4347.02 segundos

Modelo guardado exitosamente en: /content/drive/MyDrive/Proyecto-IA3/checkpoints/transfer_learning_model.pt

Intentando extraer métricas adicionales del archivo CSV...

Métricas finales del entrenamiento:
Época: 30.0
mAP@50: 0.9882
mAP@50-95: 0.7624
Precisión: 0.9601
Recall: 0.9506
Métricas extraídas exitosamente del archivo CSV


# **4. Fine-tuning (Fase 2)**

In [ ]:
# Función para Fine-tuning
def fine_tuning_phase(base_model_path):
    """Fase 2: Fine-tuning con modelo completo descongelado."""

    # Configurar la clave API de WandB
    try:
        with open('/content/drive/MyDrive/Proyecto-IA3/wandb_api_key.txt', 'r') as f:
            os.environ["WANDB_API_KEY"] = f.read().strip()
    except FileNotFoundError:
        print("Archivo wandb_api_key.txt no encontrado. Usando clave de respaldo.")
        os.environ["WANDB_API_KEY"] = "30a7a34eb742653f47d9208966cbed1dc6f31e2d"

    # Inicializar WandB para fine-tuning
    wandb.init(
        project="school-violence-detection-yolo11_V2",
        name=f"fine_tuning_{datetime.now().strftime('%Y%m%d_%H%M%S')}",
        config={
            "phase": "fine_tuning",
            "architecture": "YOLOv11n",
            "frozen_layers": 0,
            "epochs": 50,
            "batch_size": 16,
            "learning_rate": 0.0001,
        }
    )

    # Cargar el modelo de Transfer Learning
    print(f"Cargando modelo desde: {base_model_path}")
    model = YOLO(base_model_path)

    # Contador para rastrear épocas
    current_epoch = [0]

    # Callback para imprimir métricas por época
    def on_val_end(validator):
        """Imprime métricas después de cada validación (por época)."""
        current_epoch[0] += 1

        # Obtener métricas del validator
        metrics = validator.metrics

        # Para YOLOv11, las métricas están en el objeto metrics
        if hasattr(metrics, 'box'):
            print(f"\n=== Época {current_epoch[0]}/50 ===")
            print(f"  Precisión: {metrics.box.mp:.4f}")
            print(f"  Recall: {metrics.box.mr:.4f}")
            print(f"  mAP@50: {metrics.box.map50:.4f}")
            print(f"  mAP@50-95: {metrics.box.map:.4f}")
        else:
            print(f"\n=== Época {current_epoch[0]}/50 ===")
            print("  Métricas aún no disponibles")

    # Configuración de hiperparámetros para fine-tuning
    finetune_params = {
        'data': f'{DATASET_PATH}/data2.yaml',
        'epochs': 50,
        'batch': 16,
        'imgsz': 640,
        'freeze': None,  # No congelar capas en fine-tuning
        'optimizer': 'AdamW',
        'lr0': 0.0001,  # Learning rate más bajo para fine-tuning
        'lrf': 0.01,
        'momentum': 0.937,
        'weight_decay': 0.0005,
        'warmup_epochs': 3,
        'warmup_momentum': 0.8,
        'warmup_bias_lr': 0.1,
        'box': 7.5,
        'cls': 0.5,
        'dfl': 1.5,
        'save': True,
        'save_period': 5,
        'cache': False,
        'device': 0,
        'workers': 4,
        'project': f'{MODELS_PATH}/fine_tuning',
        'name': 'exp',
        'exist_ok': True,
        'verbose': True,
        'seed': 42,
        'deterministic': True,
        'single_cls': True,
        'cos_lr': True,
        'patience': 15,  # Mayor paciencia para fine-tuning
        'amp': True,
        'plots': True,
        'resume': False,
    }

    print("Iniciando Fine-tuning con modelo completo...")
    print("Todas las capas están descongeladas")

    # Registrar callback
    model.add_callback('on_val_end', on_val_end)

    # Medir tiempo de entrenamiento
    start_time = time.time()

    # Entrenar
    results = model.train(**finetune_params)

    # Calcular tiempo de entrenamiento
    training_time = time.time() - start_time

    # Guardar el modelo final
    model.save(f'{MODELS_PATH}/yolo11_people_final.pt')

    # Evaluar el modelo
    metrics = model.val()

    # Guardar métricas
    finetune_metrics = {
        'mAP50': float(metrics.box.map50),
        'mAP50-95': float(metrics.box.map),
        'precision': float(metrics.box.mp),
        'recall': float(metrics.box.mr),
        'training_time': training_time,
    }

    with open(f'{RESULTS_PATH}/fine_tuning_metrics.json', 'w') as f:
        json.dump(finetune_metrics, f, indent=4)

    # Visualizar curvas de entrenamiento
    plot_training_curves(results, phase='fine_tuning')

    # Limpiar memoria
    gc.collect()
    torch.cuda.empty_cache()

    wandb.finish()

    return model, results, metrics




# Ejecutar Fine-tuning
print("\nIniciando fase de Fine-tuning...")
final_model, finetune_results, finetune_metrics = fine_tuning_phase(f'{CHECKPOINTS_PATH}/transfer_learning_model.pt')

# Mostrar métricas finales
if os.path.exists(f'{RESULTS_PATH}/fine_tuning_metrics.json'):
    with open(f'{RESULTS_PATH}/fine_tuning_metrics.json', 'r') as f:
        metrics = json.load(f)
        print("\nMétricas finales de Fine-tuning:")
        print(f"mAP@50: {metrics['mAP50']:.4f}")
        print(f"mAP@50-95: {metrics['mAP50-95']:.4f}")
        print(f"Precisión: {metrics['precision']:.4f}")
        print(f"Recall: {metrics['recall']:.4f}")
        print(f"Tiempo de entrenamiento: {metrics['training_time']:.2f} segundos")

# Verificar si el modelo final se guardó correctamente
if os.path.exists(f'{MODELS_PATH}/yolo11_people_final.pt'):
    print(f"\nModelo final guardado exitosamente en: {MODELS_PATH}/yolo11_people_final.pt")
else:
    print("\nAdvertencia: No se encontró el archivo del modelo final")

# Comparar métricas de Transfer Learning vs Fine-tuning
print("\nComparación de resultados:")
print("=" * 50)

if os.path.exists(f'{RESULTS_PATH}/transfer_learning_metrics.json') and os.path.exists(f'{RESULTS_PATH}/fine_tuning_metrics.json'):
    with open(f'{RESULTS_PATH}/transfer_learning_metrics.json', 'r') as f:
        transfer_metrics = json.load(f)
    with open(f'{RESULTS_PATH}/fine_tuning_metrics.json', 'r') as f:
        finetune_metrics = json.load(f)

    print("Métrica          Transfer Learning  Fine-tuning   Mejora")
    print("-" * 55)

    for metric in ['mAP50', 'mAP50-95', 'precision', 'recall']:
        transfer_value = transfer_metrics[metric]
        finetune_value = finetune_metrics[metric]
        improvement = ((finetune_value - transfer_value) / transfer_value) * 100

        print(f"{metric:<15} {transfer_value:.4f}           {finetune_value:.4f}       {improvement:+.2f}%")

    print("=" * 50)

# **5. Evaluación Completa y Métricas**

In [ ]:
# Evaluación exhaustiva con métricas adicionales
def comprehensive_evaluation(model, dataset_path):
    """Realiza una evaluación completa con métricas binarias y curvas."""

    # Evaluar en validación y test
    val_metrics = model.val(data=f'{dataset_path}/data.yaml', split='val')
    test_metrics = model.val(data=f'{dataset_path}/data.yaml', split='test')

    # Calcular métricas binarias (persona/no persona)
    def compute_binary_metrics(split='test'):
        """Calcula matriz de confusión, sensibilidad, especificidad, TPR, FPR."""
        test_images_path = os.path.join(dataset_path, 'images', split)
        test_labels_path = os.path.join(dataset_path, 'labels', split)

        y_true = []
        y_pred = []
        y_scores = []

        for img_file in os.listdir(test_images_path):
            if not img_file.endswith('.jpg'):
                continue
            img_path = os.path.join(test_images_path, img_file)
            label_path = os.path.join(test_labels_path, img_file.replace('.jpg', '.txt'))

            # Etiqueta real: 1 si hay al menos un bounding box, 0 si no
            true_label = 1 if os.path.exists(label_path) and os.path.getsize(label_path) > 0 else 0
            y_true.append(true_label)

            # Predicción
            results = model(img_path, verbose=False)
            pred_label = 1 if len(results[0].boxes) > 0 else 0
            y_pred.append(pred_label)

            # Score de confianza
            scores = results[0].boxes.conf.cpu().numpy() if len(results[0].boxes) > 0 else [0.0]
            y_scores.append(max(scores) if scores else 0.0)

        # Calcular métricas
        cm = confusion_matrix(y_true, y_pred)
        tn, fp, fn, tp = cm.ravel()
        sensitivity = tp / (tp + fn) if (tp + fn) > 0 else 0
        specificity = tn / (tn + fp) if (tn + fp) > 0 else 0
        tpr = sensitivity
        fpr = fp / (fp + tn) if (fp + tn) > 0 else 0

        # Curvas ROC y Precision-Recall
        fpr_curve, tpr_curve, _ = roc_curve(y_true, y_scores)
        roc_auc = auc(fpr_curve, tpr_curve)
        precision_curve, recall_curve, _ = precision_recall_curve(y_true, y_scores)
        pr_auc = auc(recall_curve, precision_curve)

        return {
            'confusion_matrix': cm.tolist(),
            'sensitivity': sensitivity,
            'specificity': specificity,
            'tpr': tpr,
            'fpr': fpr,
            'roc_auc': roc_auc,
            'pr_auc': pr_auc,
            'fpr_curve': fpr_curve.tolist(),
            'tpr_curve': tpr_curve.tolist(),
            'precision_curve': precision_curve.tolist(),
            'recall_curve': recall_curve.tolist(),
        }

    test_binary_metrics = compute_binary_metrics(split='test')

    # Consolidar resultados
    eval_results = {
        'validation': {
            'mAP50': float(val_metrics.box.map50),
            'mAP50-95': float(val_metrics.box.map),
            'precision': float(val_metrics.box.mp),
            'recall': float(val_metrics.box.mr),
            'f1_score': float(2 * val_metrics.box.mp * val_metrics.box.mr / (val_metrics.box.mp + val_metrics.box.mr + 1e-6)),
        },
        'test': {
            'mAP50': float(test_metrics.box.map50),
            'mAP50-95': float(test_metrics.box.map),
            'precision': float(test_metrics.box.mp),
            'recall': float(test_metrics.box.mr),
            'f1_score': float(2 * test_metrics.box.mp * test_metrics.box.mr / (test_metrics.box.mp + test_metrics.box.mr + 1e-6)),
            'binary_metrics': test_binary_metrics,
        }
    }

    # Guardar resultados
    with open(f'{RESULTS_PATH}/final_evaluation_metrics.json', 'w') as f:
        json.dump(eval_results, f, indent=4)

    # Visualizar métricas
    plot_evaluation_metrics(eval_results)

    return eval_results

def plot_evaluation_metrics(eval_results):
    """Visualiza métricas de evaluación, incluyendo matriz de confusión y curvas."""
    # Gráfico de métricas estándar
    metrics = ['mAP50', 'mAP50-95', 'precision', 'recall', 'f1_score']
    val_values = [eval_results['validation'][m] for m in metrics]
    test_values = [eval_results['test'][m] for m in metrics]

    fig, ax = plt.subplots(figsize=(12, 8))
    x = np.arange(len(metrics))
    width = 0.35
    ax.bar(x - width/2, val_values, width, label='Validation', alpha=0.8)
    ax.bar(x + width/2, test_values, width, label='Test', alpha=0.8)
    ax.set_ylabel('Score')
    ax.set_title('Model Evaluation Metrics')
    ax.set_xticks(x)
    ax.set_xticklabels(metrics)
    ax.legend()
    ax.grid(True, alpha=0.3)
    for i, v in enumerate(val_values):
        ax.text(i - width/2, v + 0.01, f'{v:.3f}', ha='center')
    for i, v in enumerate(test_values):
        ax.text(i + width/2, v + 0.01, f'{v:.3f}', ha='center')
    plt.savefig(f'{RESULTS_PATH}/evaluation_metrics.png', dpi=300, bbox_inches='tight')
    plt.show()

    # Visualizar matriz de confusión
    cm = np.array(eval_results['test']['binary_metrics']['confusion_matrix'])
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=['No Person', 'Person'], yticklabels=['No Person', 'Person'])
    plt.title('Confusion Matrix (Test Set)')
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.savefig(f'{RESULTS_PATH}/confusion_matrix.png', dpi=300, bbox_inches='tight')
    plt.show()

    # Visualizar curvas ROC y Precision-Recall
    fpr = np.array(eval_results['test']['binary_metrics']['fpr_curve'])
    tpr = np.array(eval_results['test']['binary_metrics']['tpr_curve'])
    roc_auc = eval_results['test']['binary_metrics']['roc_auc']

    precision = np.array(eval_results['test']['binary_metrics']['precision_curve'])
    recall = np.array(eval_results['test']['binary_metrics']['recall_curve'])
    pr_auc = eval_results['test']['binary_metrics']['pr_auc']

    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))
    ax1.plot(fpr, tpr, label=f'ROC Curve (AUC = {roc_auc:.3f})')
    ax1.plot([0, 1], [0, 1], 'k--')
    ax1.set_xlabel('False Positive Rate')
    ax1.set_ylabel('True Positive Rate')
    ax1.set_title('ROC Curve')
    ax1.legend()
    ax1.grid(True)

    ax2.plot(recall, precision, label=f'PR Curve (AUC = {pr_auc:.3f})')
    ax2.set_xlabel('Recall')
    ax2.set_ylabel('Precision')
    ax2.set_title('Precision-Recall Curve')
    ax2.legend()
    ax2.grid(True)

    plt.tight_layout()
    plt.savefig(f'{RESULTS_PATH}/roc_pr_curves.png', dpi=300, bbox_inches='tight')
    plt.show()

# Ejecutar evaluación completa
evaluation_results = comprehensive_evaluation(final_model, DATASET_PATH)

#  Análisis de falsos positivos/negativos

In [ ]:
# Análisis de falsos positivos/negativos
def analyze_false_predictions(model, dataset_path, split='test', num_samples=10):
    """Analiza y visualiza falsos positivos y negativos."""
    test_images_path = os.path.join(dataset_path, 'images', split)
    test_labels_path = os.path.join(dataset_path, 'labels', split)

    image_files = [f for f in os.listdir(test_images_path) if f.endswith('.jpg')]
    false_positives = []
    false_negatives = []

    for img_file in image_files:
        img_path = os.path.join(test_images_path, img_file)
        label_path = os.path.join(test_labels_path, img_file.replace('.jpg', '.txt'))

        true_label = 1 if os.path.exists(label_path) and os.path.getsize(label_path) > 0 else 0
        results = model(img_path, verbose=False)
        pred_label = 1 if len(results[0].boxes) > 0 else 0

        if true_label == 0 and pred_label == 1:
            false_positives.append(img_file)
        elif true_label == 1 and pred_label == 0:
            false_negatives.append(img_file)

    # Visualizar muestras
    def visualize_samples(title, samples, max_samples=5):
        if not samples:
            print(f"No {title} encontrados.")
            return
        samples = random.sample(samples, min(max_samples, len(samples)))
        fig, axes = plt.subplots(1, len(samples), figsize=(5 * len(samples), 5))
        if len(samples) == 1:
            axes = [axes]
        for idx, img_file in enumerate(samples):
            img_path = os.path.join(test_images_path, img_file)
            img = cv2.imread(img_path)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            results = model(img_path, verbose=False)
            for box in results[0].boxes:
                x1, y1, x2, y2 = map(int, box.xyxy[0])
                cv2.rectangle(img, (x1, y1), (x2, y2), (255, 0, 0), 2)
            axes[idx].imshow(img)
            axes[idx].axis('off')
            axes[idx].set_title(img_file)
        plt.suptitle(title)
        plt.savefig(f'{RESULTS_PATH}/{title.lower().replace(" ", "_")}.png', dpi=300, bbox_inches='tight')
        plt.show()

    visualize_samples("False Positives", false_positives)
    visualize_samples("False Negatives", false_negatives)

    return {
        'false_positives': len(false_positives),
        'false_negatives': len(false_negatives),
        'total_images': len(image_files),
        'fp_rate': len(false_positives) / len(image_files),
        'fn_rate': len(false_negatives) / len(image_files),
    }

false_predictions = analyze_false_predictions(final_model, DATASET_PATH)
print("False Predictions Analysis:")
print(f"False Positives: {false_predictions['false_positives']} ({false_predictions['fp_rate']:.3f})")
print(f"False Negatives: {false_predictions['false_negatives']} ({false_predictions['fn_rate']:.3f})")

# **6. Análisis de Rendimiento e Inferencia**

In [ ]:
# Análisis de rendimiento en inferencia
def analyze_inference_performance(model, test_images_path, num_samples=100):
    """Analiza el rendimiento del modelo en inferencia."""
    image_files = [f for f in os.listdir(test_images_path) if f.endswith('.jpg')]
    sample_files = random.sample(image_files, min(num_samples, len(image_files)))

    inference_times = []
    preprocessing_times = []
    postprocessing_times = []

    for img_file in sample_files:
        img_path = os.path.join(test_images_path, img_file)
        start_time = time.time()
        results = model(img_path, verbose=False)
        end_time = time.time()
        total_time = (end_time - start_time) * 1000
        inference_times.append(total_time)
        if hasattr(results[0], 'speed'):
            preprocessing_times.append(results[0].speed['preprocess'])
            postprocessing_times.append(results[0].speed['postprocess'])

    performance_stats = {
        'inference': {
            'mean': np.mean(inference_times),
            'std': np.std(inference_times),
            'min': np.min(inference_times),
            'max': np.max(inference_times),
            'median': np.median(inference_times)
        },
        'fps': 1000 / np.mean(inference_times),
        'preprocessing': {
            'mean': np.mean(preprocessing_times) if preprocessing_times else 0,
            'std': np.std(preprocessing_times) if preprocessing_times else 0,
        },
        'postprocessing': {
            'mean': np.mean(postprocessing_times) if postprocessing_times else 0,
            'std': np.std(postprocessing_times) if postprocessing_times else 0,
        }
    }

    plt.figure(figsize=(10, 6))
    plt.hist(inference_times, bins=30, edgecolor='black', alpha=0.7)
    plt.axvline(np.mean(inference_times), color='red', linestyle='dashed', linewidth=2, label=f'Mean: {np.mean(inference_times):.2f}ms')
    plt.axvline(np.median(inference_times), color='green', linestyle='dashed', linewidth=2, label=f'Median: {np.median(inference_times):.2f}ms')
    plt.xlabel('Inference Time (ms)')
    plt.ylabel('Frequency')
    plt.title('Inference Time Distribution')
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.savefig(f'{RESULTS_PATH}/inference_time_distribution.png', dpi=300, bbox_inches='tight')
    plt.show()

    return performance_stats

test_images_path = f'{DATASET_PATH}/images/test'
performance_stats = analyze_inference_performance(final_model, test_images_path)
print(f"Average FPS: {performance_stats['fps']:.2f}")

# **7. Generación de Reporte Final**

In [ ]:
# Generar reporte completo
def generate_final_report(model, transfer_metrics, finetune_metrics, eval_results, performance_stats, false_predictions):
    """Genera un reporte completo del entrenamiento y evaluación."""
    report = f"""
# YOLOv11 People Detection - Training Report
Generated on: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}

## Model Information
- Architecture: YOLOv11n
- Input Size: 640x640
- Number of Classes: 1 (person)
- Framework: Ultralytics YOLOv11
- Dropout: Added 0.3 in head
- Augmentations: None

## Training Process

### Phase 1: Transfer Learning
- Epochs: 30
- Frozen Layers: 10
- Learning Rate: 0.001
- Final Metrics:
  - mAP@50: {transfer_metrics['mAP50']:.4f}
  - mAP@50-95: {transfer_metrics['mAP50-95']:.4f}
  - Precision: {transfer_metrics['precision']:.4f}
  - Recall: {transfer_metrics['recall']:.4f}

### Phase 2: Fine-tuning
- Epochs: 70
- Frozen Layers: 0 (all unfrozen)
- Learning Rate: 0.0001
- Final Metrics:
  - mAP@50: {finetune_metrics['mAP50']:.4f}
  - mAP@50-95: {finetune_metrics['mAP50-95']:.4f}
  - Precision: {finetune_metrics['precision']:.4f}
  - Recall: {finetune_metrics['recall']:.4f}

## Final Model Evaluation

### Validation Set
- mAP@50: {eval_results['validation']['mAP50']:.4f}
- mAP@50-95: {eval_results['validation']['mAP50-95']:.4f}
- Precision: {eval_results['validation']['precision']:.4f}
- Recall: {eval_results['validation']['recall']:.4f}
- F1-Score: {eval_results['validation']['f1_score']:.4f}

### Test Set
- mAP@50: {eval_results['test']['mAP50']:.4f}
- mAP@50-95: {eval_results['test']['mAP50-95']:.4f}
- Precision: {eval_results['test']['precision']:.4f}
- Recall: {eval_results['test']['recall']:.4f}
- F1-Score: {eval_results['test']['f1_score']:.4f}
- Sensitivity: {eval_results['test']['binary_metrics']['sensitivity']:.4f}
- Specificity: {eval_results['test']['binary_metrics']['specificity']:.4f}
- TPR: {eval_results['test']['binary_metrics']['tpr']:.4f}
- FPR: {eval_results['test']['binary_metrics']['fpr']:.4f}
- ROC AUC: {eval_results['test']['binary_metrics']['roc_auc']:.4f}
- PR AUC: {eval_results['test']['binary_metrics']['pr_auc']:.4f}

## False Predictions Analysis
- False Positives: {false_predictions['false_positives']} ({false_predictions['fp_rate']:.3f})
- False Negatives: {false_predictions['false_negatives']} ({false_predictions['fn_rate']:.3f})

## Inference Performance
- Average Inference Time: {performance_stats['inference']['mean']:.2f}ms
- Standard Deviation: {performance_stats['inference']['std']:.2f}ms
- FPS: {performance_stats['fps']:.2f}
- Min Time: {performance_stats['inference']['min']:.2f}ms
- Max Time: {performance_stats['inference']['max']:.2f}ms
- Preprocessing Time: {performance_stats['preprocessing']['mean']:.2f}ms
- Postprocessing Time: {performance_stats['postprocessing']['mean']:.2f}ms

## Model Files
- Final Model: {MODELS_PATH}/yolo11_people_final.pt
- Transfer Learning Checkpoint: {CHECKPOINTS_PATH}/transfer_learning_model.pt

## Notes
- Use WandB Sweeps for hyperparameter optimization (suggested ranges: lr0=[1e-4, 1e-2], batch=[8, 32], weight_decay=[1e-5, 1e-3]).
- Test model under adverse conditions (low light, occlusions) before deployment.
"""

    report_path = f'{RESULTS_PATH}/final_training_report.md'
    with open(report_path, 'w') as f:
        f.write(report)

    print(f"Reporte final guardado en: {report_path}")
    return report

# Generar reporte final
final_report = generate_final_report(
    final_model,
    transfer_metrics,
    finetune_metrics,
    evaluation_results,
    performance_stats,
    false_predictions
)
print(final_report)

# **8. Exportación y Preparación para Producción**

In [ ]:
# Exportar modelo para diferentes plataformas
def export_model_formats(model):
    """Exporta el modelo a diferentes formatos para despliegue."""
    export_formats = {
        'onnx': {'format': 'onnx', 'imgsz': 640, 'simplify': True},
        'torchscript': {'format': 'torchscript', 'imgsz': 640},
        'tflite': {'format': 'tflite', 'imgsz': 640},
    }

    exported_paths = {}
    for format_name, params in export_formats.items():
        try:
            print(f"Exportando a {format_name}...")
            export_path = model.export(**params)
            exported_paths[format_name] = export_path
            print(f"✓ Exportado exitosamente a: {export_path}")
        except Exception as e:
            print(f"✗ Error exportando a {format_name}: {str(e)}")

    return exported_paths

# Exportar modelo
exported_models = export_model_formats(final_model)

# Guardar información de despliegue
deployment_info = {
    'model_architecture': 'YOLOv11n',
    'input_size': 640,
    'num_classes': 1,
    'class_names': ['person'],
    'export_date': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
    'exported_formats': exported_models,
    'performance_metrics': {
        'mAP50': evaluation_results['test']['mAP50'],
        'mAP50-95': evaluation_results['test']['mAP50-95'],
        'fps': performance_stats['fps'],
        'sensitivity': evaluation_results['test']['binary_metrics']['sensitivity'],
        'specificity': evaluation_results['test']['binary_metrics']['specificity'],
    }
}

with open(f'{MODELS_PATH}/deployment_info.json', 'w') as f:
    json.dump(deployment_info, f, indent=4)

print("\n¡Entrenamiento completado exitosamente!")
print(f"Modelo final: {MODELS_PATH}/yolo11_people_final.pt")

# **9. CONCLUSIONES**